In [1]:
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [20]:
import json
from nltk.translate import nist_score, bleu_score

helsinki_src = ['en', 'de', 'ru', 'lt', 'uk', 'fr']
yandex_src = ['ba',]

In [5]:
print(nist_score.sentence_nist(['Hello how are you', 'Hey how are you'], 'Hi how are you')) # the higher the beter
print(bleu_score.sentence_bleu(['Hello how are you'], 'Hi how are you')) # from 0 to 1

3.210584896448878
0.6905103676445132


In [6]:
def read_json(filename):
    with open(filename) as f:
        return json.load(f)
    
def write_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        
def get_tgt_langs(data):
    tgt_langs = list()
    for q in data['questions']:
        for l in q['question']:
            if l['language'] not in tgt_langs:
                tgt_langs.append(l['language'])
        break
    return tgt_langs

def get_representations(data, id_, lang):
    representations = list()
    for q in data['questions']:
        if q['id'] == id_:
            for l in q['question']:
                if l['language'] == lang:
                    representations.append(l['string'])
            break
    return representations

In [7]:
test = read_json("../data/qald_test_wikidata.json")

# Helsinki NLP

In [14]:
# todo src-tgt storage
bleu_dict = dict()
nist_dict = dict()

for lang in helsinki_src:
    translated_test = read_json(f"../data/translated/qald_train_wikidata-helsinki-{lang}.json")
    tgt_langs = get_tgt_langs(translated_test)
    
    bleu_dict[lang] = dict()
    nist_dict[lang] = dict()
    
    for tgt in tgt_langs:
        bleu_dict[lang][tgt] = list()
        nist_dict[lang][tgt] = list()
    
    for q in test['questions']:
        for tgt in tgt_langs:
            representations = get_representations(translated_test, q['id'], tgt)
            original = get_representations(test, q['id'], tgt)
            if len(original) and len(representations):
                bleu_dict[lang][tgt].append(bleu_score.sentence_bleu(original, representations[0]))
                nist_dict[lang][tgt].append(nist_score.sentence_nist(original, representations[0]))

In [15]:
for lang in helsinki_src:
    translated_test = read_json(f"../data/translated/qald_train_wikidata-helsinki-{lang}.json")
    tgt_langs = get_tgt_langs(translated_test)
    
    for tgt in tgt_langs:
        print('----------', lang, tgt, '--------------')
        if len(bleu_dict[lang][tgt]) > 0:
            print("BLEU:", sum(bleu_dict[lang][tgt])/len(bleu_dict[lang][tgt]))
            print("NIST:", sum(nist_dict[lang][tgt])/len(nist_dict[lang][tgt]))
        else:
            print("No intersections between orig and translated")

---------- en de --------------
BLEU: 0.08642620301475021
NIST: 2.0092406538360383
---------- en ru --------------
BLEU: 0.055689327882144306
NIST: 1.834447265611166
---------- en fr --------------
BLEU: 0.06714250095197337
NIST: 2.032730434401108
---------- de en --------------
BLEU: 0.0800307914963027
NIST: 1.9213823727059245
---------- de fr --------------
BLEU: 0.08224967483439095
NIST: 2.292955492898556
---------- ru en --------------
BLEU: 0.0676708993438789
NIST: 1.8573814943078344
---------- ru fr --------------
BLEU: 0.07136961096882107
NIST: 2.0548540916880635
---------- lt de --------------
BLEU: 0.0893717027259498
NIST: 2.0077126589930328
---------- lt ru --------------
BLEU: 0.05151551563039392
NIST: 1.8071935354036497
---------- lt fr --------------
BLEU: 0.04835950054536907
NIST: 1.8211274076552066
---------- uk en --------------
BLEU: 0.0721771764063748
NIST: 1.9477471515662248
---------- uk de --------------
BLEU: 0.08710618095866396
NIST: 1.9319946343298742
----------

# Yandex

In [21]:
# todo src-tgt storage
bleu_dict = dict()
nist_dict = dict()

for lang in yandex_src:
    translated_test = read_json(f"../data/translated/qald_test_wikidata-yandex-{lang}.json")
    tgt_langs = get_tgt_langs(translated_test)
    
    bleu_dict[lang] = dict()
    nist_dict[lang] = dict()
    
    for tgt in tgt_langs:
        bleu_dict[lang][tgt] = list()
        nist_dict[lang][tgt] = list()
    
    for q in test['questions']:
        for tgt in tgt_langs:
            representations = get_representations(translated_test, q['id'], tgt)
            original = get_representations(test, q['id'], tgt)
            if len(original) and len(representations):
                bleu_dict[lang][tgt].append(bleu_score.sentence_bleu(original, representations[0]))
                nist_dict[lang][tgt].append(nist_score.sentence_nist(original, representations[0]))

In [22]:
for lang in yandex_src:
    translated_test = read_json(f"../data/translated/qald_test_wikidata-yandex-{lang}.json")
    tgt_langs = get_tgt_langs(translated_test)
    
    for tgt in tgt_langs:
        print('----------', lang, tgt, '--------------')
        if len(bleu_dict[lang][tgt]) > 0:
            print("BLEU:", sum(bleu_dict[lang][tgt])/len(bleu_dict[lang][tgt]))
            print("NIST:", sum(nist_dict[lang][tgt])/len(nist_dict[lang][tgt]))
        else:
            print("No intersections between orig and translated")

---------- ba en --------------
BLEU: 0.5501396718042613
NIST: 3.676277616715481
---------- ba de --------------
BLEU: 0.5088845838080437
NIST: 3.4857228063101258
---------- ba ru --------------
BLEU: 0.6274610268352264
NIST: 3.993843593666274
---------- ba fr --------------
BLEU: 0.4894012916585365
NIST: 3.444668611894536
